In [9]:
import pandas
with open("nba_2013.csv", "r") as csvfile:
    nba_raw = pandas.read_csv(csvfile)
# Replace NaN values with zeros.
nba = nba_raw.fillna(0)
# Convert strings to NaN and drop.
nba = nba.convert_objects(convert_numeric=True).dropna()
# The names of the columns in the data.
print("nba.columns.values:", nba.columns.values)
nba.head(5)

nba.columns.values: ['player' 'pos' 'age' 'bref_team_id' 'g' 'gs' 'mp' 'fg' 'fga' 'fg.' 'x3p'
 'x3pa' 'x3p.' 'x2p' 'x2pa' 'x2p.' 'efg.' 'ft' 'fta' 'ft.' 'orb' 'drb'
 'trb' 'ast' 'stl' 'blk' 'tov' 'pf' 'pts' 'season' 'season_end']


C:\Users\kishan.chadalawada\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [24]:
import math
selected_player = nba[nba["player"] == "LeBron James"].iloc[0]
distance_columns = ['age', 'g', 'gs', 'mp', 'fg' ,'fga', 'fg.' ,'x3p'
 ,'x3pa' ,'x3p.' ,'x2p', 'x2pa' ,'x2p.', 'efg.' ,'ft', 'fta', 'ft.' ,'orb', 'drb',
 'trb', 'ast' ,'stl' ,'blk', 'tov', 'pf' ,'pts']
def euclidean_distance(row):
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
    return math.sqrt(inner_value)
lebron_distance = nba.apply(euclidean_distance, axis=1)
print("lebron_distance[:5]:\n", lebron_distance[:5])

lebron_distance[:5]:
 0    3475.792868
1    3148.395020
2    3161.567361
3    1189.554979
4    3216.773098
dtype: float64


In [25]:
nba_numeric = nba[distance_columns]
nba_numeric.head(5)
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()
nba_normalized.head(5)

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,-0.835906,0.384886,-0.862207,-0.435088,-0.738401,-0.768505,0.325957,-0.700282,-0.716608,0.120520,...,-0.151926,0.260690,-0.129462,-0.013116,-0.645220,-0.468056,0.061410,-0.667650,0.226515,-0.734621
1,-1.550487,1.095711,-0.187863,-0.045011,-0.581271,-0.649215,0.667749,-0.778936,-0.829601,-1.390497,...,-0.522588,1.387883,0.187020,0.565852,-0.530733,0.020680,1.065446,-0.013760,1.363938,-0.534801
2,0.116868,-0.010016,-0.457600,-0.308035,-0.290291,-0.405214,0.833763,-0.778936,-0.829601,-1.390497,...,-0.250457,0.743773,0.283340,0.436083,-0.568895,-0.439307,0.385292,-0.524113,0.029924,-0.328603
3,0.355062,0.779789,1.599148,1.465144,1.577804,1.590172,0.238067,1.737992,1.430256,1.027130,...,0.575320,-0.383420,0.462221,0.216475,1.033919,-0.123066,-0.683520,1.182380,0.423107,1.729123
4,-0.359519,0.108454,0.149309,-0.319180,-0.331028,-0.475703,1.087666,-0.778936,-0.822068,-1.390497,...,0.673851,0.614951,0.138859,0.291341,-0.553630,-0.468056,0.709175,-0.141348,1.139262,-0.400878


In [31]:
from scipy.spatial import distance
# Fill in NA values in nba_normalized.
nba_normalized.fillna(0, inplace=True)
# Find the normalized vector for lebron james.
lebron_normalized = nba_normalized[nba["player"] == "LeBron James"]
# Find the distance between lebron james and everyone else.
euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)
distance_frame = pandas.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})
#distance_frame.sort("dist", inplace=True)
second_smallest = distance_frame.iloc[1]["idx"]
most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]
print("most_similar_to_lebron:", most_similar_to_lebron)

most_similar_to_lebron: Steven Adams


In [32]:
import random
from numpy.random import permutation
# Randomly shuffle the index of nba.
random_indices = permutation(nba.index)
# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items).
test_cutoff = math.floor(len(nba)/3)
# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba.loc[random_indices[1:test_cutoff]]
# Generate the train set with the rest of the data.
train = nba.loc[random_indices[test_cutoff:]]

In [36]:
from sklearn.neighbors import KNeighborsRegressor
# The columns that we will be making predictions with.
x_columns = ['age', 'g', 'gs', 'mp', 'fg' ,'fga', 'fg.' ,'x3p'
 ,'x3pa' ,'x3p.' ,'x2p', 'x2pa' ,'x2p.', 'efg.' ,'ft', 'fta', 'ft.' ,'orb', 'drb',
 'trb', 'ast' ,'stl' ,'blk', 'tov', 'pf' ]
# The column that we want to predict.
y_column = ['pts']
# Create the knn model.
knn = KNeighborsRegressor(n_neighbors=5)
# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])
# Make predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])
print("predictions[:5]:\n", predictions[:5])
# ###8: Computing error
# Now that we know our predictions, we can compute the error involved. We can compute <a href = "http://en.wikipedia.org/wiki/Mean_squared_error">mean squared error</a>. The formula is $\frac{1}{n}\sum_{i=1}^{n}(\hat{y_{i}} - y_{i})^{2}$.
# ####Instructions
# Compute the mean squared error between actual and predictions. Assign the result to mse.

predictions[:5]:
 [[593. ]
 [980.4]
 [714.4]
 [110.6]
 [598. ]]


In [37]:
actual = test[y_column]
mse = (((predictions - actual) ** 2).sum()) / len(predictions)
print("actual[:20]:\n", actual[:20])
print("mse:", mse)

actual[:20]:
       pts
466   603
195  1053
371   677
135   145
418   622
257    15
335   429
430   703
0     171
392   350
69    274
181   404
141  1096
123  1791
23    280
374   350
159  1041
299    73
124   717
160  1457
mse: pts    3136.358994
dtype: float64
